In [3]:
%pip install scikit-learn

    pandas (>=1.0.*)
            ~~~~~~^
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd

df = pd.read_csv('/projects/diffusion_bootcamp/datasets/ibmaml/HI-Small_Trans.csv')
df.head()


,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
0,2022/09/01 00:20,10,8000EBD30,10,8000EBD30,3697.34,US Dollar,3697.34,US Dollar,Reinvestment,0
1,2022/09/01 00:20,3208,8000F4580,1,8000F5340,0.01,US Dollar,0.01,US Dollar,Cheque,0
2,2022/09/01 00:00,3209,8000F4670,3209,8000F4670,14675.57,US Dollar,14675.57,US Dollar,Reinvestment,0
3,2022/09/01 00:02,12,8000F5030,12,8000F5030,2806.97,US Dollar,2806.97,US Dollar,Reinvestment,0
4,2022/09/01 00:06,10,8000F5200,10,8000F5200,36682.97,US Dollar,36682.97,US Dollar,Reinvestment,0


# Explore Data

In [2]:
df.columns

Index(['Timestamp', 'From Bank', 'Account', 'To Bank', 'Account.1',
       'Amount Received', 'Receiving Currency', 'Amount Paid',
       'Payment Currency', 'Payment Format', 'Is Laundering'],
      dtype='object')

In [7]:
print(len(set(df['Account'])))

print(df.shape)


print(len(set(df['Account.1'])))



496995
(5078345, 11)
420636


In [22]:
# Feature Engineering 1: identical account activity
df["Identitcal Account Activity"] = ((df["Account"] == df["Account.1"]) & (df["From Bank"] == df["To Bank"])).astype(int)


In [49]:
# ((df["Account"] == df["Account.1"]) & (df["From Bank"] == df["To Bank"]) & (df["Payment Format"] != "Reinvestment"))

filtered_df = df[
    (df['Account'] == df['Account.1']) &
    (df['From Bank'] == df['To Bank']) &
    (df['Payment Format'] != 'Reinvestment')
]

filtered_df

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering,Same Account Activity,Same Bank Activity,Identitcal Account Activity,Datetime,SecondsSinceEpoch
105,2022/09/01 00:26,12,800043C00,12,800043C00,755.250000,US Dollar,755.250000,US Dollar,Cheque,0,1,1,1,2022-09-01 00:26:00,1661991960
106,2022/09/01 00:20,12,800043C00,12,800043C00,876.160000,US Dollar,876.160000,US Dollar,Credit Card,0,1,1,1,2022-09-01 00:20:00,1661991600
109,2022/09/01 00:20,10,800056660,10,800056660,4461.810000,US Dollar,4461.810000,US Dollar,Cheque,0,1,1,1,2022-09-01 00:20:00,1661991600
110,2022/09/01 00:03,10,800056660,10,800056660,1134.320000,US Dollar,1134.320000,US Dollar,Credit Card,0,1,1,1,2022-09-01 00:03:00,1661990580
111,2022/09/01 00:18,10,800056660,10,800056660,726.940000,US Dollar,726.940000,US Dollar,Cash,0,1,1,1,2022-09-01 00:18:00,1661991480
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5078269,2022/09/10 23:35,126,813B6E021,126,813B6E021,0.213793,Bitcoin,0.213793,Bitcoin,Bitcoin,0,1,1,1,2022-09-10 23:35:00,1662852900
5078270,2022/09/10 23:59,126,813B6E021,126,813B6E021,0.121724,Bitcoin,0.121724,Bitcoin,Bitcoin,0,1,1,1,2022-09-10 23:59:00,1662854340
5078271,2022/09/10 23:38,15,813B765F1,15,813B765F1,0.003384,Bitcoin,0.003384,Bitcoin,Bitcoin,0,1,1,1,2022-09-10 23:38:00,1662853080
5078272,2022/09/10 23:37,15,813B765F1,15,813B765F1,0.006543,Bitcoin,0.006543,Bitcoin,Bitcoin,0,1,1,1,2022-09-10 23:37:00,1662853020


In [42]:
# Feature Engineering 2: Timestamp normalization
df['Datetime'] = pd.to_datetime(df['Timestamp'], format='%Y/%m/%d %H:%M')
df["SecondsSinceEpoch"] = (df['Datetime'].astype(int) // 10**9)


In [50]:
# Final DF
feature_engineered_df = df[["SecondsSinceEpoch", "From Bank", "Account","To Bank","Account.1", "Identitcal Account Activity", "Amount Received", "Receiving Currency", "Amount Paid", "Payment Currency", "Payment Format", "Is Laundering"]]
feature_engineered_df

,SecondsSinceEpoch,From Bank,Account,To Bank,Account.1,Identitcal Account Activity,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
0,1661991600,10,8000EBD30,10,8000EBD30,1,3697.340000,US Dollar,3697.340000,US Dollar,Reinvestment,0
1,1661991600,3208,8000F4580,1,8000F5340,0,0.010000,US Dollar,0.010000,US Dollar,Cheque,0
2,1661990400,3209,8000F4670,3209,8000F4670,1,14675.570000,US Dollar,14675.570000,US Dollar,Reinvestment,0
3,1661990520,12,8000F5030,12,8000F5030,1,2806.970000,US Dollar,2806.970000,US Dollar,Reinvestment,0
4,1661990760,10,8000F5200,10,8000F5200,1,36682.970000,US Dollar,36682.970000,US Dollar,Reinvestment,0
...,...,...,...,...,...,...,...,...,...,...,...,...
5078340,1662854220,54219,8148A6631,256398,8148A8711,0,0.154978,Bitcoin,0.154978,Bitcoin,Bitcoin,0
5078341,1662852900,15,8148A8671,256398,8148A8711,0,0.108128,Bitcoin,0.108128,Bitcoin,Bitcoin,0
5078342,1662853920,154365,8148A6771,256398,8148A8711,0,0.004988,Bitcoin,0.004988,Bitcoin,Bitcoin,0
5078343,1662853560,256398,8148A6311,256398,8148A8711,0,0.038417,Bitcoin,0.038417,Bitcoin,Bitcoin,0


In [46]:
# Group by 'From Bank' and 'To Bank' and sum the 'Is Laundering' column
result = df.groupby(['From Bank', 'To Bank'])['Is Laundering'].sum().reset_index()

# Rename the column for clarity
result = result.rename(columns={'Is Laundering': 'Total Laundering Count'})

# Sort the results by 'Total Laundering Count' in descending order
result = result.sort_values(by='Total Laundering Count', ascending=False)

result

df['From Bank'].nunique()
df['To Bank'].nunique()

,From Bank,To Bank,Total Laundering Count
72982,15231,23691,27
103670,23691,15231,20
161188,49365,119,18
187865,148350,148350,16
7923,119,49365,15
...,...,...,...
96132,21611,231706,0
96133,21611,231786,0
96134,21611,232555,0
96135,21611,232583,0


# Data sampling for Fanout

In [47]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

class BankAccountMerger(BaseEstimator, TransformerMixin):
    def __init__(self, source_bank_col, source_account_col, dest_bank_col, dest_account_col):
        self.source_bank_col = source_bank_col
        self.source_account_col = source_account_col
        self.dest_bank_col = dest_bank_col
        self.dest_account_col = dest_account_col

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        # Convert to string and then concatenate
        X['Source'] = X[self.source_bank_col].astype(str) + '_' + X[self.source_account_col].astype(str)
        X['Destination'] = X[self.dest_bank_col].astype(str) + '_' + X[self.dest_account_col].astype(str)
        return X.drop([self.source_bank_col, self.source_account_col, 
                       self.dest_bank_col, self.dest_account_col], axis=1)

class SourceAccountSampler(BaseEstimator, TransformerMixin):
    def __init__(self, source_col, sample_size=100, random_state=None):
        self.source_col = source_col
        self.sample_size = sample_size
        self.random_state = random_state

    def fit(self, X, y=None):
        unique_sources = X[self.source_col].unique()
        np.random.seed(self.random_state)
        self.sampled_sources = np.random.choice(unique_sources, 
                                                size=min(self.sample_size, len(unique_sources)), 
                                                replace=False)
        return self

    def transform(self, X):
        return X[X[self.source_col].isin(self.sampled_sources)]

# Create the pipeline
pipeline = Pipeline([
    ('merge_accounts', BankAccountMerger('From Bank', 'Account', 'To Bank', 'Account.1')),
    ('sample_sources', SourceAccountSampler('Source', sample_size=100, random_state=42))
])

class CurrencyConverter(BaseEstimator, TransformerMixin):
    def __init__(self, amount_received_col, receiving_currency_col, amount_paid_col, payment_currency_col):
        self.amount_received_col = amount_received_col
        self.receiving_currency_col = receiving_currency_col
        self.amount_paid_col = amount_paid_col
        self.payment_currency_col = payment_currency_col
        
        # Define conversion rates (you should update these regularly)
        self.conversion_rates = {
            'US Dollar': 1.36,
            'Euro': 1.51,
            'Yuan': 0.19,
            'Rupee': 0.016,
            'Ruble': 0.014,
            'Canadian Dollar': 1.0,
            'Australian Dollar': 0.92,
            'Brazil Real': 0.25,
            'Swiss Franc': 1.60,
            'Shekel': 0.37,
            'Bitcoin': 35000,
            'Yen': 0.0095,
            'UK Pound': 1.80
        }

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        
        # Convert Amount Received to CAD
        X['Amount Received CAD'] = X.apply(lambda row: row[self.amount_received_col] * self.conversion_rates.get(row[self.receiving_currency_col], 1), axis=1)
        
        # Convert Amount Paid to CAD
        X['Amount Paid CAD'] = X.apply(lambda row: row[self.amount_paid_col] * self.conversion_rates.get(row[self.payment_currency_col], 1), axis=1)
        
        return X

class PatternIdAssigner(BaseEstimator, TransformerMixin):
    def __init__(self, source_col, destination_col, timestamp_col, window_days=7, max_degree=5):
        self.source_col = source_col
        self.destination_col = destination_col
        self.timestamp_col = timestamp_col
        self.window_days = window_days
        self.max_degree = max_degree

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        
        # Ensure timestamp column is datetime type
        X[self.timestamp_col] = pd.to_datetime(X[self.timestamp_col])
        
        # Sort by source and timestamp
        X = X.sort_values([self.source_col, self.timestamp_col])
        
        pattern_ids = []
        pattern_id = 0
        current_source = None
        start_time = None
        destination_count = 0
        
        for _, row in X.iterrows():
            if current_source != row[self.source_col]:
                # New source, start a new pattern
                pattern_id += 1
                current_source = row[self.source_col]
                start_time = row[self.timestamp_col]
                destination_count = 0
            elif (row[self.timestamp_col] - start_time).days > self.window_days:
                # Exceeded time window, start a new pattern
                pattern_id += 1
                start_time = row[self.timestamp_col]
                destination_count = 0
            elif destination_count >= self.max_degree:
                # Exceeded max degree, start a new pattern
                pattern_id += 1
                start_time = row[self.timestamp_col]
                destination_count = 0
            
            # Count destination if it's not the same as the source
            if row[self.destination_col] != row[self.source_col]:
                destination_count += 1
            
            pattern_ids.append(pattern_id)
        
        X['Pattern_ID'] = pattern_ids
        return X

# Create the updated pipeline
pipeline = Pipeline([
    ('merge_accounts', BankAccountMerger('From Bank', 'Account', 'To Bank', 'Account.1')),
    ('sample_sources', SourceAccountSampler('Source', sample_size=100, random_state=42)),
    ('convert_currency', CurrencyConverter('Amount Received', 'Receiving Currency', 'Amount Paid', 'Payment Currency')),
    ('assign_pattern_id', PatternIdAssigner('Source', 'Destination', 'Timestamp', window_days=7, max_degree=16))
])

class DataFrameReshaper(BaseEstimator, TransformerMixin):
    def __init__(self, max_degree, currencies):
        self.max_degree = max_degree
        self.currencies = currencies

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Group by Pattern_ID
        grouped = X.groupby('Pattern_ID')

        # Create destination columns
        dest_columns = {f'Destination_{i+1}': grouped.apply(lambda x: x.sort_values('Amount Paid CAD', ascending=False)['Amount Paid CAD'].iloc[i] if i < len(x) else 0).astype(int)
                        for i in range(self.max_degree)}

        # Create currency columns
        currency_columns = {currency: grouped['Receiving Currency'].apply(lambda x: currency in x.values).astype(int)
                            for currency in self.currencies}

        # Calculate other metrics
        metrics = {
            'number_transactions_above_9k_cad': grouped['Amount Paid CAD'].apply(lambda x: (x > 9000).sum()),
            'avg_transaction_value_in_cad': grouped['Amount Paid CAD'].mean(),
            'transaction_value_variance_in_cad': grouped['Amount Paid CAD'].var(),
            'variance_from_10k_cad': grouped['Amount Paid CAD'].apply(lambda x: ((x - 10000)**2).sum()),
            'min_day_to_max_day_range': grouped['Timestamp'].apply(lambda x: (x.max() - x.min()).days),
            'avg_transaction_frequency_in_days': grouped['Timestamp'].apply(lambda x: (x.max() - x.min()).days / len(x) if len(x) > 1 else 0),
            'transaction_frequency_variance_in_days': grouped['Timestamp'].apply(lambda x: pd.Series(x).diff().dt.total_seconds() / (24*3600)).var(),
            'Is FanOut': grouped['Is Laundering'].first()  # Assuming 'Is Laundering' is already a boolean column
        }

        # Combine all parts
        result = pd.concat([
            grouped['Source'].first(),
            pd.DataFrame(dest_columns),
            pd.DataFrame(currency_columns),
            pd.DataFrame(metrics)
        ], axis=1)

        return result.reset_index()

# List of currencies
currencies = ['US Dollar', 'Euro', 'Yuan', 'Rupee', 'Ruble', 'Canadian Dollar',
              'Australian Dollar', 'Brazil Real', 'Swiss Franc', 'Shekel',
              'Bitcoin', 'Yen', 'UK Pound']

# Create the updated pipeline
pipeline = Pipeline([
    ('merge_accounts', BankAccountMerger('From Bank', 'Account', 'To Bank', 'Account.1')),
    ('sample_sources', SourceAccountSampler('Source', sample_size=100, random_state=42)),
    ('convert_currency', CurrencyConverter('Amount Received', 'Receiving Currency', 'Amount Paid', 'Payment Currency')),
    ('assign_pattern_id', PatternIdAssigner('Source', 'Destination', 'Timestamp', window_days=7, max_degree=16)),
    ('reshape_dataframe', DataFrameReshaper(max_degree=16, currencies=currencies))
])

# Apply the pipeline
max_degree=16
result_df = pipeline.fit_transform(df)




/tmp/ipykernel_2721/3554092288.py:174: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  'transaction_frequency_variance_in_days': grouped['Timestamp'].apply(lambda x: pd.Series(x).diff().dt.total_seconds() / (24*3600)).var(),


In [48]:
result_df.head(20)

,Pattern_ID,Source,Destination_1,Destination_2,Destination_3,Destination_4,Destination_5,Destination_6,Destination_7,Destination_8,...,Yen,UK Pound,number_transactions_above_9k_cad,avg_transaction_value_in_cad,transaction_value_variance_in_cad,variance_from_10k_cad,min_day_to_max_day_range,avg_transaction_frequency_in_days,transaction_frequency_variance_in_days,Is FanOut
0,1,10441_80B948670,6376,5369,0,0,0,0,0,0,...,0,0,0,5.873107e+03,5.063292e+05,3.456882e+07,6,3.000000,1.66884,0
1,2,10_802144AF0,2118007,2118007,2118007,2118007,639683,449285,449285,449285,...,0,0,13,6.485703e+05,7.507306e+11,1.894381e+13,5,0.294118,1.66884,0
2,3,10_802144AF0,2118007,2118007,2118007,449285,449285,449285,22306,22306,...,0,0,9,6.491257e+05,8.189731e+11,1.391048e+13,1,0.083333,1.66884,0
3,4,112497_805451700,927,18,0,0,0,0,0,0,...,0,0,0,4.729762e+02,4.133341e+05,1.819417e+08,0,0.000000,1.66884,0
4,5,11296_80D9430D0,15,0,0,0,0,0,0,0,...,0,0,0,1.566720e+01,NaN,9.968690e+07,0,0.000000,1.66884,0
5,6,115693_805F4F100,2915,1053,0,0,0,0,0,0,...,0,0,0,1.984601e+03,1.732288e+06,1.302255e+08,3,1.500000,1.66884,0
6,7,116214_80BF9CF10,15,0,0,0,0,0,0,0,...,0,0,0,1.591200e+01,NaN,9.968201e+07,0,0.000000,1.66884,0
7,8,117_80E60EDC0,28344,5501,5343,5343,4152,4039,2644,881,...,0,0,1,3.290279e+03,4.347238e+07,1.549397e+09,4,0.222222,1.66884,0
8,9,117_80E60EDC0,5343,5343,5343,5343,881,881,235,235,...,0,0,0,1.553376e+03,5.166960e+06,1.219032e+09,3,0.187500,1.66884,0
9,10,117_80E60EDC0,5343,2644,881,881,235,173,173,107,...,0,0,0,1.304942e+03,3.372462e+06,6.284395e+08,0,0.000000,1.66884,0


In [50]:
import os

# Get current working directory
current_directory = os.getcwd()

print("Current Working Directory:", current_directory)

Current Working Directory: /fs01/home/ws_aabboud/diffusion_model_bootcamp/deloitte_team/single_table_synthesis


In [52]:
result_df.to_csv('/fs01/home/ws_aabboud/diffusion_model_bootcamp/deloitte_team/single_table_synthesis/data/processed_data/flattened_sample.csv', index=False)